In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd

In [2]:
data = {"Title": [], 
        "book_link": [], 
        "img_link": [], 
        "Rating": [], 
        "Price": [], 
        "is Available": [], 
        "Type": [], 
        "Product UPC": [], 
        "Price (excl. tax)": [], 
        "Price (incl. tax)": [], 
        "Tax": [], 
        "Availability": [], 
        "Reviews": []}

In [3]:
for i in tqdm(range(1, 51), desc="Scraping pages", position=0, colour="green", leave = False):
    url = f"https://books.toscrape.com/catalogue/page-{i}.html"
    res = requests.get(url).text
    soup = BeautifulSoup(res, "lxml")

    for book in soup.find_all("li", class_ = "col-xs-6 col-sm-4 col-md-3 col-lg-3"):
        data["img_link"].append(book.find("img").get("src"))
        book_id = book.find_all("a")[-1].get("href")
        data["Title"].append(book.find_all("a")[-1].get("title"))
        data["Rating"].append(book.find("p").get("class")[-1])
        data["Price"].append(book.find("p", class_ = "price_color").text[2:])
        data["is Available"].append(True if book.find("p", class_ = "instock availability").text.strip() == "In stock" else False)

        link = "https://books.toscrape.com/catalogue/" + book_id
        about = requests.get(link).text
        soup2 = BeautifulSoup(about, "lxml")
        data["book_link"].append(link)

        data["Type"].append(soup2.find("ul", class_ = "breadcrumb").find_all("a")[2].text)
        
        table = soup2.find("table", class_ = "table table-striped").find_all("td")
        data["Product UPC"].append(table[0].text)
        data["Price (excl. tax)"].append(table[2].text[2:])
        data["Price (incl. tax)"].append(table[3].text[2:])
        data["Tax"].append(table[4].text[2:])
        data["Availability"].append(table[5].text)
        data["Reviews"].append(table[6].text)

In [4]:
df = pd.DataFrame(data)
df

,Title,book_link,img_link,Rating,Price,is Available,Type,Product UPC,Price (excl. tax),Price (incl. tax),Tax,Availability,Reviews
0,A Light in the Attic,https://books.toscrape.com/catalogue/a-light-i...,../media/cache/2c/da/2cdad67c44b002e7ead0cc356...,Three,51.77,True,Poetry,a897fe39b1053632,51.77,51.77,0.00,In stock (22 available),0
1,Tipping the Velvet,https://books.toscrape.com/catalogue/tipping-t...,../media/cache/26/0c/260c6ae16bce31c8f8c95dadd...,One,53.74,True,Historical Fiction,90fa61229261140a,53.74,53.74,0.00,In stock (20 available),0
2,Soumission,https://books.toscrape.com/catalogue/soumissio...,../media/cache/3e/ef/3eef99c9d9adef34639f51066...,One,50.10,True,Fiction,6957f44c3847a760,50.10,50.10,0.00,In stock (20 available),0
3,Sharp Objects,https://books.toscrape.com/catalogue/sharp-obj...,../media/cache/32/51/3251cf3a3412f53f339e42cac...,Four,47.82,True,Mystery,e00eb4fd7b871a48,47.82,47.82,0.00,In stock (20 available),0
4,Sapiens: A Brief History of Humankind,https://books.toscrape.com/catalogue/sapiens-a...,../media/cache/be/a5/bea5697f2534a2f86a3ef27b5...,Five,54.23,True,History,4165285e1663650f,54.23,54.23,0.00,In stock (20 available),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,https://books.toscrape.com/catalogue/alice-in-...,../media/cache/96/ee/96ee77d71a31b7694dac6855f...,One,55.53,True,Classics,cd2a2a70dd5d176d,55.53,55.53,0.00,In stock (1 available),0
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",https://books.toscrape.com/catalogue/ajin-demi...,../media/cache/09/7c/097cb5ecc6fb3fbe1690cf0cb...,Four,57.06,True,Sequential Art,bfd5e1701c862ac3,57.06,57.06,0.00,In stock (1 available),0
997,A Spy's Devotion (The Regency Spies of London #1),https://books.toscrape.com/catalogue/a-spys-de...,../media/cache/1b/5f/1b5ff86f3c75e51e24c573d3f...,Five,16.97,True,Historical Fiction,19fec36a1dfb4c16,16.97,16.97,0.00,In stock (1 available),0
998,1st to Die (Women's Murder Club #1),https://books.toscrape.com/catalogue/1st-to-di...,../media/cache/2b/41/2b4161c5b72a4ae386b644682...,One,53.98,True,Mystery,f684a82adc49f011,53.98,53.98,0.00,In stock (1 available),0


In [5]:
df.to_csv("books.csv", index = False)